In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### Chargement des données

In [ ]:
def load(name):
    df = pd.read_json(name, lines=True).fillna(pd.NA).sort_values('time').reset_index(drop=True)
    df = df.assign(time=(df.time - df.time[0]))
    df = df.assign(step=df.time.diff())
    df = df.set_index('time')
    return df

In [ ]:
delay_df = load('../metrics-delay.json')
fast_df= load('../metrics-fast.json')

## Répartition des espacements de dates

In [ ]:
bins = np.arange(0 ,2e10, 3e8)

In [ ]:
delay_df.step.hist(log=True, bins=bins)

In [ ]:
fast_df.step.hist(log=True, bins=bins)

## Répartition par type

In [ ]:
[ fast_df[['name']].assign(count=1).groupby('name').count().plot.bar(log=True),
 delay_df[['name']].assign(count=1).groupby('name').count().plot.bar(log=True)]

In [ ]:
fast_df[['name']].assign(count=1).groupby('name').count()

In [ ]:
(delay_df[['name']].assign(count=1).groupby('name').count() - fast_df[['name']].assign(count=1).groupby('name').count()).rename(columns={'count': 'count_diff'})

On a plus 25 appels directs de moins en mode `delay`. Ce sont eux qui peuvent empécher d'optimiser. 

Y a-t-il un lien entre les -25 **direct** et les 26 **callSite** ?

# Direct : Les primitives

In [ ]:
fast_prim = fast_df[fast_df.name == 'direct']
delay_prim = delay_df[delay_df.name == 'direct']

In [ ]:
fast_prim.groupby('type').count()

In [ ]:
delay_prim.groupby('type').count()

> Beaucoup de primitives

# callSite

In [ ]:
callSite_fast = fast_df[['name','apply']][fast_df.name == 'callSite'].groupby('apply').count().sort_values('name', ascending=False)
callSite_fast

In [ ]:
callSite_fast.plot()

In [ ]:
callSite_fast_sum = callSite_fast.assign(count_sum=callSite_fast.cumsum()/10)
callSite_fast_sum.plot()

Sur {{ 2+2 }} Il n'y a que la moitié des **callSite** à optimiser

In [ ]:
callSite_fast_sum.query('count_sum < 4000')

# calledLambda

In [ ]:
df = fast_df[['name','lambda']][fast_df.name == 'calledLambda'].groupby('lambda').count().sort_values('name', ascending=False)
df

In [ ]:
df_sum = df.assign(count_sum=df.cumsum()/5)
df_sum.plot()

# pushStack

In [ ]:
fast_df[fast_df.name == 'pushStack'].groupby('expr').count().sort_values('')

In [ ]:
df = fast_df[['name','expr']][fast_df.name == 'pushStack'].groupby('expr').count().sort_values('name', ascending=False)
df

In [ ]:
df_sum = df.assign(count_sum=df.cumsum()/30)
df_sum.plot()

# Conclusion
On voit bien, comme prévu, que certains éléments sont utilisés plus que d'autres. On a les éléments pour un compilateur...